# Exploratory Data Analysis **(EDA)** & Pre-processing steps

## `AZIDAS` dataset:  Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).

This notebook performs Exploratory Data Analysis and pre-processing (data standarization and cleaning) of the `AZIDAS` dataset containing demographics for the general population of Germany.

# 00. Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
try:
    from tqdm import tqdm
except:
    !pip install tqdm
    from tqdm import tqdm    
%matplotlib inline


### my utils

from eda_utils import *

---

# 01. Import dataset
We know that the separator is a semicolon (;), and the first row contains the column names. Also, one of the columns 'LNR' is really the `ID` number, and should not be considered a standard "feature". 

In [2]:
#plot raw header to inspect dataset
!head -n 2 ../arvato_data/Udacity_AZDIAS_052018.csv

LNR;AGER_TYP;AKT_DAT_KL;ALTER_HH;ALTER_KIND1;ALTER_KIND2;ALTER_KIND3;ALTER_KIND4;ALTERSKATEGORIE_FEIN;ANZ_HAUSHALTE_AKTIV;ANZ_HH_TITEL;ANZ_KINDER;ANZ_PERSONEN;ANZ_STATISTISCHE_HAUSHALTE;ANZ_TITEL;ARBEIT;BALLRAUM;CAMEO_DEU_2015;CAMEO_DEUG_2015;CAMEO_INTL_2015;CJT_GESAMTTYP;CJT_KATALOGNUTZER;CJT_TYP_1;CJT_TYP_2;CJT_TYP_3;CJT_TYP_4;CJT_TYP_5;CJT_TYP_6;D19_BANKEN_ANZ_12;D19_BANKEN_ANZ_24;D19_BANKEN_DATUM;D19_BANKEN_DIREKT;D19_BANKEN_GROSS;D19_BANKEN_LOKAL;D19_BANKEN_OFFLINE_DATUM;D19_BANKEN_ONLINE_DATUM;D19_BANKEN_ONLINE_QUOTE_12;D19_BANKEN_REST;D19_BEKLEIDUNG_GEH;D19_BEKLEIDUNG_REST;D19_BILDUNG;D19_BIO_OEKO;D19_BUCH_CD;D19_DIGIT_SERV;D19_DROGERIEARTIKEL;D19_ENERGIE;D19_FREIZEIT;D19_GARTEN;D19_GESAMT_ANZ_12;D19_GESAMT_ANZ_24;D19_GESAMT_DATUM;D19_GESAMT_OFFLINE_DATUM;D19_GESAMT_ONLINE_DATUM;D19_GESAMT_ONLINE_QUOTE_12;D19_HANDWERK;D19_HAUS_DEKO;D19_KINDERARTIKEL;D19_KONSUMTYP;D19_KONSUMTYP_MAX;D19_KOSMETIK;D19_LEBENSMITTEL;D19_LETZTER_KAUF_BRANCHE;D19_LOTTO;D19_NAHRUNGSERGAENZUNG;D19_RATGEBE

In [3]:
# count features
!head -n 1 ../arvato_data/Udacity_AZDIAS_052018.csv | tr ";" "\n" | wc -l

366


In [4]:
# show features
!head -n 1 ../arvato_data/Udacity_AZDIAS_052018.csv | tr ";" "\n" | sort

AGER_TYP
AKT_DAT_KL
ALTER_HH
ALTER_KIND1
ALTER_KIND2
ALTER_KIND3
ALTER_KIND4
ALTERSKATEGORIE_FEIN
ALTERSKATEGORIE_GROB
ANREDE_KZ
ANZ_HAUSHALTE_AKTIV
ANZ_HH_TITEL
ANZ_KINDER
ANZ_PERSONEN
ANZ_STATISTISCHE_HAUSHALTE
ANZ_TITEL
ARBEIT
BALLRAUM
CAMEO_DEU_2015
CAMEO_DEUG_2015
CAMEO_INTL_2015
CJT_GESAMTTYP
CJT_KATALOGNUTZER
CJT_TYP_1
CJT_TYP_2
CJT_TYP_3
CJT_TYP_4
CJT_TYP_5
CJT_TYP_6
D19_BANKEN_ANZ_12
D19_BANKEN_ANZ_24
D19_BANKEN_DATUM
D19_BANKEN_DIREKT
D19_BANKEN_GROSS
D19_BANKEN_LOKAL
D19_BANKEN_OFFLINE_DATUM
D19_BANKEN_ONLINE_DATUM
D19_BANKEN_ONLINE_QUOTE_12
D19_BANKEN_REST
D19_BEKLEIDUNG_GEH
D19_BEKLEIDUNG_REST
D19_BILDUNG
D19_BIO_OEKO
D19_BUCH_CD
D19_DIGIT_SERV
D19_DROGERIEARTIKEL
D19_ENERGIE
D19_FREIZEIT
D19_GARTEN
D19_GESAMT_ANZ_12
D19_GESAMT_ANZ_24
D19_GESAMT_DATUM
D19_GESAMT_OFFLINE_DATUM
D19_GESAMT_ONLINE_DATUM
D19_GESAMT_ONLINE_QUOTE_12
D19_HANDWERK
D19_HAUS_DEKO
D19_KINDERARTIKEL
D19_KONSUMTYP
D19_KONSUMTYP_MAX
D19_KOSMETIK
D19_LEBENSMITTEL
D19_LETZTER_KAUF_BRANCHE
D19_LOTTO
D19_NAH

I have a machine with 32GB of `RAM`, so the dataset can be loaded and pre-processed in memory. Hence `low_memory = False`, allows for faster loads and better column type inference.

In [ ]:
%%time
azdias = pd.read_csv('../arvato_data/Udacity_AZDIAS_052018.csv', sep=';', low_memory = False, index_col = 'LNR') 
azdias

---

# 02. Missing features

## 02.01. Intro



Before exploring the values contained in the cells, it is essential to understand how many missing values are present. I.e. values that were **not entered**, now "unknown" from the perspective of the data curator, the latter will come later.

In [ ]:
%%time
empty_cells = pd.isnull(azdias).sum()*100.0/azdias.shape[0]
empty_cells = empty_cells.sort_values(ascending=False)
empty_cells[:50].plot(figsize=(20,3),kind='bar') # bar plot of first 50 most missing features

(Above) a barplot showing 50 most missing features, ordered by the percentage of missing values.

(Below) Histogram showing that most features have `NAs` less than 20% missing values:

In [ ]:
plt.hist(empty_cells,bins=15)
plt.plot()

For subsequent steps in pre-processing, I will be establishing a cutoff threshold for a feature removal: called `missingness_threshold_percentage`. 
> If a feature is missing more or equal to `missingness_threshold_percentage`% of its entries, I am removing that feature from subsequent analysis. Since the data is describing general population, and high values of missingness might introduce a lot of spuriousness to the analysis. 


In [ ]:
missingness_threshold_percentage=30

In [ ]:
np.sum(empty_cells>=missingness_threshold_percentage)

For now, with 30% cut-off threshold I would remove **6 features**. But I will postpose this step for now to identify other entries annotated as "missing/unknown".

## 02.02. Undefined features
I am utilizing provided matadata file `DIAS Attributes - Values 2017.xlsx` to extract the possible value types for defined features (columns in `azidas` dataset).

Due to the specific formatting, a forward-fill function is applied `ffill`. 

Upon manual inspection, the values for "unknown" are sometimes encoded as -1, sometimes as 0 or 9, and sometimes as both -1,9 or 0. But there are features in which for example 0 does not mean unknown. And there are features marked with string `...` indicating a **float values**, as opposed to a categorical one.

In this step, I am going to count number of "unknown" entries per feature. The idea is that "missing entries" (point 02.01) and unknown entries "02.02" might have different sources, but inevitabely lead to the same conclusions for the sake of the analysis -> unavailable data.

I am going to use the keyword "unknown" to match the values in the column "meaning", to extract a dataframe of attributes and its values defining unknown entries:

And now I will change values in loaded dataset: `azdias` that match the missing values in specified attributes to `NANs`:

In [ ]:
metadata_attributes = r"../arvato_data/DIAS Attributes - Values 2017.xlsx"
azdias,missing_metadata_annotations,not_present_features = unknown_to_nan(azdias, metadata_attributes , 'unknown|no transactions known') 

In [ ]:
print("{} dataset features lacking metadata information, while there are {} features not used by provided dataset. ".\
      format(  len(missing_metadata_annotations), len(not_present_features) ) )

## 02.03. Features renaming & discussion
To investigate, let's take **for example** features that have "D19" in their name from both dataset and metadata information:

In [ ]:
[x for x in  not_present_features if "D19" in x]

In [ ]:
[x for x in  missing_metadata_annotations if "D19" in x]

It seems that there are examples, like:

- `D19_TELKO_MOBILE` (present in `AZIDAS` dataset), but lacking metadata entry, however the `D19_TELKO_MOBILE_RZ` is defined in metadata.

- `D19_BANKEN_GROSS` in the dataset, and `D19_BANKEN_GROSS_RZ` in metadata

for this particular example, it is perhaps worth assuming that these features refer to the same concept. 

It is particularly important to reduce the number of data fields that lack metadata information, as we need to define a concept of missingness. For example `D19_TELKO_MOBILE_RZ` in provided metadata file defines the fields as: "transactional activity based on the product group MOBILE COMMUNICATION", and is described by several categories:

```
0	no transaction known
1	Multibuyer 0-12 months
2	Doublebuyer 0-12 months
3	Singlebuyer 0-12 months
4	Multi-/Doublebuyer 13-24 months
5	Singlebuyer 13-24 months
6	Buyer > 24 months
7	Prospects > 24 months
```

We might wonder what this `0` means really - this would be a question to the data curator. 


If 0 label (no known transation) is interpreted as a lack of knowledge, i.e. missing value, then this value **in reality** could be really one of the 1-7 classes. Thus 0 would be associated with uncertainity, i.e. missing value. Then we can either drop it, or try to impute it based on other features. But if we're certain that it indicates no transations, then it is a valid category in itself. 

One thing is that `D19_TELKO_MOBILE` feature does not hold any `NAs` values in the loaded dataframe:

In [ ]:
np.sum(np.isnan(azdias['D19_TELKO_MOBILE']))

In [ ]:
list(zip( np.unique(azdias['D19_TELKO_MOBILE'], return_counts=True)[0],np.unique(azdias['D19_TELKO_MOBILE'], return_counts=True)[1] ))

while the `0` annotation is clearly the most frequently occuring one. I.e. if it were interpreted as missing, then we'd probably remove this column (feature) altogether.

In this apprach I will modify the provided metadata file: make a copy named `Amended Copy of DIAS Attributes - Values 2017.xlsx` that contains re-named features manually identified to reflect the same characteristic of the data. 

In [ ]:
np.unique(azdias['STRUKTURTYP'].dropna())

## 02.04. Missing features: correction

In [ ]:
print("{} dataset features lacking metadata information, while there are {} features not used by provided dataset. ".\
      format(  len(missing_metadata_annotations), len(not_present_features) ) )

As we see in the cell output above there are a couple of non-common features between the dataset and the provided metadata, namely
1. There are **93** feature names defined in the provided dataset that lack the exact definition in provided metadata.

2. There are 8 entries in the metadata, that were not provided for the dataset.

Whereas the (2) point is not that important (there might have been more features described, but they didn't make it to the final dataset for some reason) the first step requires some attention. If we're missing annotations for 93 features, then we're missing a source of the "unknown/undefined/NA", that could help us standardize the unknown tokens in the dataset.

I've prepared a file `azdias_corrected_features.tsv` that maps some of the column names in `azdias` dataset to metadata information stored in `DIAS Attributes - Values 2017.xlsx`. I've managed to manually correct **38** features, among them many with `D19` prefix discussed above. 

The same function can take an optinal argument `rename_columns` that takes in a tab-separated file (`.tsv`) with columns `data` and `metadata`, and maps (replaces only for complete cases) the columns in provided dataframe `azidas` from `data -> metadata`, so that now the loaded dataframe has renamed columns that match some of the identified entries in metadata.

In [ ]:
metadata_attributes = r"../arvato_data/DIAS Attributes - Values 2017.xlsx"
azdias,missing_metadata_annotations,not_present_features = unknown_to_nan(azdias, metadata_attributes , 'unknown|no transactions known',
                                                                         rename_columns='azdias_corrected_features.tsv') 

In [ ]:
np.unique(azdias['D19_GESAMT_ONLINE_QUOTE_12'].dropna(),return_counts=True)

In [ ]:
print("{} dataset features lacking metadata information, while there are {} features not used by provided dataset. ".\
      format(  len(missing_metadata_annotations), len(not_present_features) ) )

We see now that it has reduced substantially (by **38**, i.e. to 55) the number of features for which we don't have metadata information, while only 4 features in metadata remain unused.

In [ ]:
not_present_features

Unfortunately, these four features couldn't be identified back to provided dataset `Azdias`, and hance would remain unused. Perhaps `customers` dataset would include them.

Let's look back at our missingness information:

---

In [ ]:
%%time
empty_cells = pd.isnull(azdias).sum()*100.0/azdias.shape[0]
empty_cells = empty_cells.sort_values(ascending=False)
empty_cells[:50].plot(figsize=(20,3),kind='bar') # bar plot of first 50 most missing features

In [ ]:
np.sum(empty_cells>missingness_threshold_percentage)

## 02.05. Feature Missingness filtering

After identifying number of empty cells (`NaNs`, and converting unknown values to `NaNs` I'll filter (remove) features that exhibit missingnes above `missingness_threshold_percentage`

In [ ]:
%%time
features_to_drop =list( (empty_cells[empty_cells>missingness_threshold_percentage]).index.values)
azdias.drop(labels=features_to_drop,inplace=True,axis='columns')

We see that now more features have the `NA` values, previously encoded by various labels, like 0, -1, or 9. Hence an increase in overall missingness percentage. 

Plot again missingness after removal:

In [ ]:
%%time
empty_cells = pd.isnull(azdias).sum()*100.0/azdias.shape[0]
empty_cells = empty_cells.sort_values(ascending=False)
empty_cells[:50].plot(figsize=(20,3),kind='bar') # bar plot of first 50 most missing features

---
# 03. Missing rows

After we've removed highly incomplete features of our dataset, we'd have to make sure that the individuals present in the dataset (row entries) don't exhibit high missingness. 

We must remove row entries (i.e. subjects) for which a substantial portion of information is missing. I am going to apply **different missingness** threshold `row_missingness_threshold_percentage` to filter entries (rows) - **defined at 50%**. The reason is that I'll be later imputing missing values based, and believe with 50% of entries present per subjects, and a big dataset of general population, I'll be able to successfully impute missing values with features having at most `missingness_threshold_percentage` (set to 30%) missing entries

In [ ]:
%%time
empty_rows = pd.isnull(azdias).sum(axis=1)*100.0/azdias.shape[1]
empty_rows = empty_rows.sort_values(ascending=False)
empty_rows[:50].plot(figsize=(20,3),kind='bar') # bar plot of first 50 most missing rows

In [ ]:
row_missingness_threshold_percentage = 50

In [ ]:
# How many percent of rows/entries/subjects were removed?
sum(empty_rows>row_missingness_threshold_percentage)*100/azdias.shape[0]

In [ ]:
plt.hist(empty_rows,bins=15)
plt.plot()

... and we see that for majority of cases the subjects have almost all features defined.

If 50% missingness for rows is defines as a cutoff threshold, then around 11.22% of the rows/subjects are removed. 

In [ ]:
%%time
samples_to_drop =list( (empty_rows[empty_rows>row_missingness_threshold_percentage]).index.values)
azdias.drop(index=samples_to_drop,inplace=True,axis='index')

In [ ]:
%%time
empty_rows = pd.isnull(azdias).sum(axis=1)*100.0/azdias.shape[1]
empty_rows = empty_rows.sort_values(ascending=False)
empty_rows[:50].plot(figsize=(20,3),kind='bar') # bar plot of first 50 most missing features

---
# 04. Data types & Feature encoding

We know (from inspecting metadata file ` DIAS Attributes - Values 2017.xlsx`) that some features are encoded as floats, some are presented as ordinal features (where relative order matters), and some are pure categorial features, for example gender. These features need to be properly encoded.

# Feature Encoding

Features in the `Udacity_AZDIAS_052018`, and described in the provided metadata file `DIAS Attributes - Values 2017.xlsx` are encoded usually by integers, however **for the most part, the features are of ordinal or categorical nature**.



## Features lacking metadata information

First however, let's inspect the unique values the features which lack metadata information hold:

In [ ]:
# re-run previous function, to update the list of some deleted columns
metadata_attributes = r"../arvato_data/DIAS Attributes - Values 2017.xlsx"
azdias,missing_metadata_annotations,not_present_features = unknown_to_nan(azdias, metadata_attributes , 'unknown|no transactions known',
                                                                         rename_columns='azdias_corrected_features.tsv') 

print("{} dataset features lacking metadata information, while there are {} features not used by provided dataset. ".\
      format(  len(missing_metadata_annotations), len(not_present_features) ) )

Since we filtered some features based on missingness, the number of unused features for which we have metadata rose from 4 -> 31, reflecting 27 removed features, which is ok.

In [ ]:
missing_metadata_feature_info =  {}
missing_metadata_feature_info['Attribute'] = []
missing_metadata_feature_info['unique_counts'] = []
missing_metadata_feature_info['unique_vals'] = []

for f in missing_metadata_annotations:
    unique_vals =  np.unique(azdias[f].dropna() )
    
    missing_metadata_feature_info['Attribute'].append(f)
    
    missing_metadata_feature_info['unique_counts'].append(len(unique_vals))
    missing_metadata_feature_info['unique_vals'].append(unique_vals)   
    
missing_metadata_feature_info = pd.DataFrame.from_dict(missing_metadata_feature_info)
missing_metadata_feature_info.sort_values('unique_counts',ascending=False, inplace=True)
missing_metadata_feature_info

In [ ]:
missing_metadata_feature_info.shape

Here I am sorting in an `ascending` manner the the number of unique values. 

Let's discuss some of the features (in order they appear in above table):

`EINGEFUEGT_AM` [**to be removed**]:

    - is some form of year encoding. Looking up the German translation it means "entered", so most likely when a particular user was entered into a database. But we're not sure. **I am going to drop this feature** however, if I had the opportunity to talk with some of the data curators, perhaps a feature could be engineered here. For example, how many years a particular client is staying with a "us" (a particular company). This could divide clients into new/established/old. But because we don't exactly know what it means, and the "story" behind the feature, it is better to drop it 
    
- `ANZ_STATISTISCHE_HAUSHALTE` [**to be encoded as numerical**]:
    - is some for of statistic (`STATISTICSCHE`) relating to the household (`HOUSHALTE`) and its  transation activity (from `ANZ` annotation in metadata file) 
    
    
- ... (remaining features with missing metadata) [**to be encoded as categorical**]:
    - because we lack any information to the missing features, the only thing we can assume is that they represent some form of category. We cannot assume ordinal relationship, as it would introduce errors if the features represent ortogonal entities, such as choice, type, etc.. Hence all remaining features would be encoded as **categorical**.

In [ ]:
missing_metadata_feature_info['Type']= 'categorical' # categorical as default

##### dropping EINGEFUEGT_AM_idx 

### from `missing_metadata_feature_info`
EINGEFUEGT_AM_idx = missing_metadata_feature_info.loc[missing_metadata_feature_info['Attribute']=='EINGEFUEGT_AM'].index.values
missing_metadata_feature_info.drop(axis='index', inplace=True, index = EINGEFUEGT_AM_idx )


EINGEFUEGT_AM_idx = missing_metadata_feature_info.loc[missing_metadata_feature_info['Attribute']=='EINGEFUEGT_AM'].index.values
missing_metadata_feature_info.loc[missing_metadata_feature_info.Attribute=='ANZ_STATISTISCHE_HAUSHALTE', 'Type'] = 'numerical'

### from AZDIAS
try:
    azdias.drop(columns='EINGEFUEGT_AM',inplace=True)
except KeyError:
    print("EINGEFUEGT_AM already deleted from AZDIAS")

    
missing_metadata_feature_info

---
## Exploring feature types

Secondly, let's visualize auto-inferred by pandas data types:

In [ ]:
plt.hist([str(x) for x in (azdias.dtypes).to_numpy().squeeze()])

Pandas tries best, but since all categorical/ordinal/numerical features are encoded as numbers, they are some form of a number: integer or float.

Interestingly there are some 5 features that are encoded as objects:

In [ ]:
azdias.loc[:,azdias.dtypes=="object" ].head()

Upon inspection, they are encoded as such, because they usually contain a mix of (**auto-inferred**) types: integer and string. 


Ok, so now let's encode the features properly:

**Manual inspection.**
I went over manually over the provided metadata informaiton: `DIAS Attributes - Values 2017` and based on descriptions and provided feature values, encoded each variable type as either:
- categorical
- numerical
- ordinal

Combining this information (stored in `metadata_feature_types.tsv`) with assumed types of features that lack the metadata annotations (dataframe `missing_metadata_feature_info`) I create a lookup table of expected feature types:


In [ ]:
missing_metadata_feature_info = missing_metadata_feature_info[['Attribute','Type']]

all_feature_types = pd.read_csv('metadata_feature_types.tsv',sep="\t").dropna()
all_feature_types = pd.concat([all_feature_types,missing_metadata_feature_info])

# must be in Azdias
all_feature_types.index = all_feature_types['Attribute']
# features that have filtered AZDIAS entry
all_feature_types = all_feature_types.loc[ set(all_feature_types['Attribute'] ).intersection( (set(azdias.columns  )))  ]

plt.hist(all_feature_types['Type'])

And now what remains is to encode these categories properly

In [ ]:
np.unique(all_feature_types['Type'],return_counts=True)

# SAVING 

In [ ]:
azdias.to_csv("../arvato_data_processed/azdias.csv") 
all_feature_types.to_csv("../arvato_data_processed/azdias_feature_types.csv") 

## TODO: remove below [EXPERIMENTS] 
Every feature

Optional! As XGBOOst can handle missing data, and perhaps it is worth applying a very loose threshold

# NEXT STEPS

# TODO [remove]


0. Feature correlation - Kruskal's lambda;

1. Feature engineering

 
    
- encode `D19_LETZTER_KAUF_BRANCHE` as string, it is a nice category, be sure to keep the NAN as NAN
- need to drop some unknown date encoded `EINGEFUEGT_AM`

> `PRAEGENDE_JUGENDJAHR`E - combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, two new variables will be created to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.

> `CAMEO_INTL_2015` - combines information on two axes: wealth and life stage. The two-digit codes will be broken by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).


2. Encode categorical values as strings, or categories
    - check pandas function for types encoding
    - prepare a table: feature -> data types: numerical, categorical or ordinal
    
    
3. prepare `customers` file in the same way
4. concatenate `azdias` and `customers` together
    - analyze intersecting columns
    - select common set of columns/features
    
5. Dimensionality reduction: 
    a. Prince + FAMD. https://github.com/MaxHalford/prince#factor-analysis-of-mixed-data-famd
    b. PCA although wrong

or mca: https://pypi.org/project/mca/